In [1]:
DATA_PATH = '/home/ubuntu/Notebooks/Project/EPIK_Solution/all_data/extracted/transformed/'
OCCURENCES_PATH = '/home/ubuntu/Notebooks/Project/EPIK_Solution/all_data/extracted/transformed/Occurrences/'
SQ_DATA_PATH = '/home/ubuntu/Notebooks/Project/EPIK_Solution/all_data/extracted/search_queries/'
AD_DATA_PATH = '/home/ubuntu/Notebooks/Project/EPIK_Solution/all_data/extracted/ads/'

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pd.set_option('display.max_columns', 50)
from learn.file_loader import load

In [3]:
data = pd.read_csv('/home/ubuntu/Notebooks/Project/EPIK_Solution/all_data/extracted/data_v1.csv')

In [4]:
data.head()

,Unnamed: 0,id,category_id,title,description,predict_sold,predict_replies,predict_views,priceValue,derivative,average,min,max,private,exchange,free,price,new,full_description,period,description_len,full_description_len,paid_ads,nb_photos,photos_surface,title_contains_phrase
0,0,1909313589,584,PROMOCJA!!! Drewniane Meble Ogrodowe Od Produc...,Oferuje bardzo ładne a przede wszystkim wygod...,0,0,39,900.0,-880.162637,70203.571429,55098.0,89643.0,1,0,0,1,1,Oferuje bardzo ładne a przede wszystkim wygod...,2017_07,247,482,0,28,10503095,1
1,1,1496614785,584,"Duży plac zabaw dla dzieci, huśtawka, zjeżdzal...",Solidny i estetycznie wykonany plac zabaw dla ...,0,0,88,2800.0,-880.162637,70203.571429,55098.0,89643.0,1,0,0,1,1,Solidny i estetycznie wykonany plac zabaw dla ...,2017_07,248,294,0,21,10577545,0
2,2,1740201284,584,"Żyworódka pierzasta - 30 cm, także sok - rośli...",ŻYWORÓDKA PIERZASTA !\nChyba najbardziej docen...,0,1,14,15.0,-880.162637,70203.571429,55098.0,89643.0,1,0,0,1,1,ŻYWORÓDKA PIERZASTA !\nChyba najbardziej docen...,2017_07,234,3977,0,15,8563736,0
3,3,479354039,584,"Trawa z rolki - PRODUCENT <CITY>, <CITY> - FOL...",TRAWNIKI ROLOWANE - SOLEC KUJAWSKI\n\nPlantacj...,0,11,238,13.0,-880.162637,70203.571429,55098.0,89643.0,0,0,0,1,1,TRAWNIKI ROLOWANE - SOLEC KUJAWSKI\n\nPlantacj...,2017_07,225,1797,0,36,20801000,0
4,4,1861434792,584,kora sosnowa ogrodowa sortowana gruba srednia ...,Nasza firma Artfili-Twój Ogród zajmuje się pro...,0,1,29,7.5,-880.162637,70203.571429,55098.0,89643.0,0,0,0,1,1,Nasza firma Artfili-Twój Ogród zajmuje się pro...,2017_07,244,1098,0,15,5087688,0


In [5]:
def get_percentage_of_sold(ad_df, category_id):
    all_ads = len(ad_df[ad_df['category_id'] == category_id]['predict_sold'])
    sold_ads = len(ad_df[(ad_df['category_id'] == category_id) & (ad_df['predict_sold'] == 1)]['predict_sold'])
    return sold_ads / all_ads

def find_categories_kinda_good(data_frame, non_zero=False):
    categories = []
    for i in range(1, 2000):
        try:
            get_percentage_of_sold(data_frame, i)
        except ValueError:
            pass
        except ZeroDivisionError:
            pass
        else:
            categories.append((i, get_percentage_of_sold(data_frame, i)))
    return list(filter(lambda x: x[1] > 0, categories)) if non_zero \
            else categories

In [8]:
good_cats = find_categories_kinda_good(data, True)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
stop_words = []
with open('learn/polish_stopwords.txt', 'r') as f:
    stop_words = [line[:-1] for line in f]
    
numbers = []
for i in range(1000):
    numbers.append(str(i))
numbers += ['00', '000', '01', '001', '011']
stop_words += numbers
stop_words += '!,",#,$,%,&,(,),*,+,-,.,/,:,;,<,=,>,?,@,[,\,\,],^,_,`,{,|,},~,\,t,\,n'.split(',')

In [11]:
vectorizer_t = CountVectorizer(max_features=500)
corpus_true = data[data['predict_sold'] == 1]['full_description']
X_t = vectorizer_t.fit_transform(corpus_true)

In [17]:
bow_t = set(filter(lambda x: x not in stop_words, vectorizer_t.get_feature_names()))

In [18]:
good_cats

[(6, 0.02531645569620253),
 (24, 0.018691588785046728),
 (25, 0.16666666666666666),
 (93, 0.06641476828549092),
 (100, 0.046877072006365206),
 (111, 0.09090909090909091),
 (140, 0.08144126357354393),
 (175, 0.05261018246167826),
 (180, 0.125),
 (188, 0.07692307692307693),
 (219, 0.03014982994760548),
 (224, 0.027286385674702193),
 (230, 0.031872464668779875),
 (231, 0.04805614221644587),
 (235, 0.05289147051261074),
 (236, 0.10411198600174978),
 (237, 0.11122881355932203),
 (238, 0.16176470588235295),
 (239, 0.11607142857142858),
 (240, 0.08095238095238096),
 (241, 0.08108108108108109),
 (244, 0.10133333333333333),
 (245, 0.13297872340425532),
 (246, 0.136),
 (247, 0.21875),
 (248, 0.15432098765432098),
 (249, 0.16981132075471697),
 (250, 0.14102564102564102),
 (251, 0.09556437071763434),
 (252, 0.13157894736842105),
 (253, 0.15853658536585366),
 (254, 0.175561797752809),
 (255, 0.13636363636363635),
 (256, 0.13043478260869565),
 (272, 0.02406417112299465),
 (279, 0.03322308563842571),

In [19]:
bow_t

{'2016',
 '5cm',
 'aparat',
 'aukcje',
 'aukcji',
 'bateria',
 'bdb',
 'białe',
 'biały',
 'bluza',
 'bluzka',
 'body',
 'brak',
 'buciki',
 'buty',
 'bądź',
 'całkowita',
 'całość',
 'cd',
 'cena',
 'cenie',
 'ceny',
 'chłopca',
 'city',
 'cm',
 'czarne',
 'czarny',
 'czas',
 'części',
 'dane',
 'dl',
 'dni',
 'dobry',
 'dobrym',
 'dodatkowe',
 'dodatkowo',
 'dole',
 'domu',
 'dotyczy',
 'duże',
 'duży',
 'dvd',
 'dysk',
 'działa',
 'dzieci',
 'dziecka',
 'dziecko',
 'dziecku',
 'dziewczynki',
 'dzień',
 'dzięki',
 'dzwonić',
 'dł',
 'długość',
 'ekran',
 'etui',
 'firmy',
 'fotelik',
 'galaxy',
 'gb',
 'gorąco',
 'gra',
 'gratis',
 'gry',
 'gwarancja',
 'gwarancji',
 'głębokość',
 'hd',
 'idealna',
 'idealne',
 'idealnie',
 'idealny',
 'idealnym',
 'ikea',
 'ilości',
 'ilość',
 'informacji',
 'instrukcja',
 'intel',
 'iphone',
 'istnieje',
 'itp',
 'jakości',
 'jazdy',
 'jednej',
 'jednym',
 'jedynie',
 'kabel',
 'karta',
 'kg',
 'kolor',
 'kolorze',
 'kombinezon',
 'komplecie',
 'ko

In [20]:
vectorizer_t.get_feature_names()

['00',
 '10',
 '100',
 '104',
 '11',
 '110',
 '12',
 '120',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '200',
 '2016',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '30',
 '32',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '42',
 '44',
 '45',
 '48',
 '50',
 '56',
 '5cm',
 '60',
 '62',
 '68',
 '70',
 '74',
 '80',
 '86',
 '90',
 '92',
 '98',
 'aby',
 'ale',
 'ani',
 'aparat',
 'aukcje',
 'aukcji',
 'bardzo',
 'bateria',
 'bdb',
 'bez',
 'białe',
 'biały',
 'bluza',
 'bluzka',
 'bo',
 'body',
 'brak',
 'buciki',
 'buty',
 'być',
 'był',
 'była',
 'były',
 'bądź',
 'będzie',
 'całkowita',
 'całość',
 'cały',
 'cd',
 'cena',
 'cenie',
 'ceny',
 'chłopca',
 'city',
 'cm',
 'co',
 'czarne',
 'czarny',
 'czas',
 'czy',
 'części',
 'dane',
 'dl',
 'dla',
 'dlatego',
 'dni',
 'do',
 'dobry',
 'dobrym',
 'dobrze',
 'dodatkowe',
 'dodatkowo',
 'dole',
 'domu',
 'dotyczy',
 'duże',
 'duży',
 'dvd',
 'dwa',
 'dwie',
 'dysk',
 'działa',
 'dzieci',
 'dziecka',
 'dziecko',

In [6]:
from learn.dataset_creator import run as run_processing

In [ ]:
data = run_processing(data)

preprocessing started.


In [30]:
x_test_vec = vectorizer_t.transform(data[data['predict_sold'] == 1]['full_description'].head(500))

In [31]:
for x in x_test_vec:
    print(x.sum())

28
11
41
22
41
4
45
31
14
20
9
15
13
11
13
80
97
15
1
4
17
12
5
20
10
9
18
10
21
8
25
17
14
43
20
8
4
15
16
7
50
4
6
6
57
15
11
31
102
15
9
7
57
37
6
6
17
13
5
3
8
4
2
3
35
1
31
12
9
15
21
10
0
25
19
7
6
34
2
8
16
7
1
12
44
11
6
24
48
14
7
120
9
9
8
8
2
18
13
23
12
3
39
14
9
7
30
30
15
6
8
6
1
23
15
10
9
4
2
10
3
9
6
80
12
14
3
14
18
6
46
6
13
25
12
7
8
3
5
4
13
0
8
11
6
6
2
9
9
31
8
3
10
4
3
28
13
34
5
15
8
12
6
5
19
35
5
3
8
31
18
4
10
10
2
9
37
6
18
21
6
3
20
13
14
19
1
23
8
46
3
16
44
5
9
15
19
4
7
12
8
31
46
1
8
13
10
20
5
10
1
6
24
13
9
8
29
11
5
2
19
2
14
2
2
10
2
17
9
8
6
7
8
9
4
6
11
8
3
2
10
20
5
68
22
9
27
14
4
9
6
6
25
7
8
30
5
38
11
34
26
13
16
3
23
7
5
2
18
8
16
6
11
5
11
3
15
4
4
8
7
41
32
17
73
2
16
4
24
16
1
7
23
4
12
24
9
8
7
2
7
2
27
8
5
16
16
5
19
7
15
12
27
13
9
16
14
3
16
3
2
4
2
7
49
2
27
10
32
2
39
6
5
17
10
10
28
9
13
40
11
8
24
10
33
4
9
19
100
7
26
23
0
19
1
132
30
6
18
31
15
28
26
10
13
50
12
57
5
3
43
8
40
4
72
16
27
7
4
13
17
20
21
29
9
15
11
11
23
22
6
27

In [32]:
x_test_vec0 = vectorizer_t.transform(data[data['predict_sold'] == 0]['full_description'].head(500))

In [33]:
for x in x_test_vec0:
    print(x.sum())

28
13
130
81
28
40
13
151
151
8
42
145
31
29
9
6
13
9
9
28
7
15
27
21
14
77
68
80
14
14
16
37
4
43
47
24
16
25
29
26
83
12
94
40
76
12
9
20
25
20
42
42
32
10
33
33
27
20
35
1
31
11
21
12
10
7
34
24
31
31
30
21
8
28
30
25
13
30
75
54
71
31
5
46
9
175
22
13
45
39
49
67
75
65
48
47
29
30
29
13
20
21
24
19
45
60
35
7
9
12
33
43
16
33
19
37
32
30
21
12
53
41
24
38
13
50
15
26
24
52
31
42
35
18
62
23
35
101
12
59
14
6
48
26
30
38
26
32
21
17
48
42
19
45
17
24
56
17
22
21
7
39
17
10
14
23
15
25
26
45
22
28
19
24
15
27
5
28
62
23
15
26
21
10
34
6
45
30
53
56
49
14
3
20
43
7
29
41
34
37
23
22
127
6
3
118
130
116
35
28
21
52
27
9
28
27
13
19
130
13
24
11
12
5
15
21
27
31
9
126
4
47
21
8
37
67
28
12
6
10
15
57
17
8
28
9
26
29
69
9
30
45
14
17
46
71
40
99
47
8
54
57
21
36
55
21
16
24
50
42
56
36
29
10
18
33
23
13
13
18
21
12
9
46
52
5
3
29
5
39
9
20
7
57
13
57
24
12
71
5
47
21
45
98
104
120
67
136
83
92
90
46
58
47
66
48
13
62
3
8
45
52
61
63
64
29
13
9
28
15
30
41
25
16
29
72
46
4
24
51
18
24
116

### Can we extract product state or good price for category???

In [22]:
#IDK, maybe

In [7]:
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import GlobalAveragePooling1D
from keras.datasets import imdb

Using TensorFlow backend.


In [40]:
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import GlobalAveragePooling1D
from keras.datasets import imdb


def create_ngram_set(input_list, ngram_value=2):
    """
    Extract a set of n-grams from a list of integers.
    >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=2)
    {(4, 9), (4, 1), (1, 4), (9, 4)}
    >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=3)
    [(1, 4, 9), (4, 9, 4), (9, 4, 1), (4, 1, 4)]
    """
    return set(zip(*[input_list[i:] for i in range(ngram_value)]))


def add_ngram(sequences, token_indice, ngram_range=2):
    """
    Augment the input list of list (sequences) by appending n-grams values.
    Example: adding bi-gram
    >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017}
    >>> add_ngram(sequences, token_indice, ngram_range=2)
    [[1, 3, 4, 5, 1337, 2017], [1, 3, 7, 9, 2, 1337, 42]]
    Example: adding tri-gram
    >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017, (7, 9, 2): 2018}
    >>> add_ngram(sequences, token_indice, ngram_range=3)
    [[1, 3, 4, 5, 1337], [1, 3, 7, 9, 2, 1337, 2018]]
    """
    new_sequences = []
    for input_list in sequences:
        new_list = input_list[:]
        for i in range(len(new_list) - ngram_range + 1):
            for ngram_value in range(2, ngram_range + 1):
                ngram = tuple(new_list[i:i + ngram_value])
                if ngram in token_indice:
                    new_list.append(token_indice[ngram])
        new_sequences.append(new_list)

    return new_sequences

In [57]:
ngram_range = 1
max_features = 20000
maxlen = 400
batch_size = 32
embedding_dims = 50
epochs = 5

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Average train sequence length: {}'.format(np.mean(list(map(len, x_train)), dtype=int)))
print('Average test sequence length: {}'.format(np.mean(list(map(len, x_test)), dtype=int)))

Loading data...
25000 train sequences
25000 test sequences
Average train sequence length: 238
Average test sequence length: 230


In [51]:
x_train[:10]

array([ [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32],
       [1, 194, 1153, 194, 8255, 78, 22

In [42]:
if ngram_range > 1:
    print('Adding {}-gram features'.format(ngram_range))
    # Create set of unique n-gram from the training set.
    ngram_set = set()
    for input_list in x_train:
        for i in range(2, ngram_range + 1):
            set_of_ngram = create_ngram_set(input_list, ngram_value=i)
            ngram_set.update(set_of_ngram)

    # Dictionary mapping n-gram token to a unique integer.
    # Integer values are greater than max_features in order
    # to avoid collision with existing features.
    start_index = max_features + 1
    token_indice = {v: k + start_index for k, v in enumerate(ngram_set)}
    indice_token = {token_indice[k]: k for k in token_indice}

    # max_features is the highest integer that could be found in the dataset.
    max_features = np.max(list(indice_token.keys())) + 1

    # Augmenting x_train and x_test with n-grams features
    x_train = add_ngram(x_train, token_indice, ngram_range)
    x_test = add_ngram(x_test, token_indice, ngram_range)
    print('Average train sequence length: {}'.format(np.mean(list(map(len, x_train)), dtype=int)))
    print('Average test sequence length: {}'.format(np.mean(list(map(len, x_test)), dtype=int)))

In [54]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 400)
x_test shape: (25000, 400)


In [55]:
x_train[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [45]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))

# we add a GlobalAveragePooling1D, which will average the embeddings
# of all words in the document
model.add(GlobalAveragePooling1D())

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Build model...


In [46]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 12s 490us/step - loss: 0.6126 - acc: 0.7430 - val_loss: 0.5051 - val_acc: 0.8225
Epoch 2/5
25000/25000 [==============================] - 12s 480us/step - loss: 0.4060 - acc: 0.8632 - val_loss: 0.3740 - val_acc: 0.8646
Epoch 3/5
25000/25000 [==============================] - 12s 482us/step - loss: 0.3062 - acc: 0.8934 - val_loss: 0.3220 - val_acc: 0.8782
Epoch 4/5
25000/25000 [==============================] - 12s 472us/step - loss: 0.2551 - acc: 0.9110 - val_loss: 0.2971 - val_acc: 0.8853
Epoch 5/5
25000/25000 [==============================] - 12s 477us/step - loss: 0.2207 - acc: 0.9248 - val_loss: 0.2843 - val_acc: 0.8878


In [8]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [83]:
min_count = 2

#docs = create_docs(data[['id','description']])
tokenizer = Tokenizer()
print('fitting 1')
tokenizer.fit_on_texts(data[data['predict_sold'] == 1]['description'])
num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])
print(num_words)

tokenizer = Tokenizer(num_words=num_words)
print('fitting 2')
tokenizer.fit_on_texts(data[data['predict_sold'] == 1]['description'])

print('texts to seqs')
docs = tokenizer.texts_to_sequences(data[data['predict_sold'] == 1]['description'])

maxlen = 256

print('padding')
docs = pad_sequences(sequences=docs, maxlen=maxlen)

fitting 1
115154
fitting 2
texts to seqs
padding


In [9]:
TRAIN_DATA = data[data['predict_sold'] == 1][['id', 'description', 'predict_sold']]

In [10]:
temp_train_data = data[data['predict_sold'] == 0][['id', 'description', 'predict_sold']]
temp_train_data_sampled = temp_train_data.sample(frac=0.4, random_state=99)

In [11]:
TRAIN_DATA = TRAIN_DATA.append(temp_train_data_sampled, 1)

In [12]:
len(TRAIN_DATA)

4231108

In [13]:
from sklearn.utils import shuffle

In [14]:
TRAIN_DATA = shuffle(TRAIN_DATA)

In [46]:
min_count = 2

#docs = create_docs(data[['id','description']])
tokenizer = Tokenizer()
print('fitting 1')
tokenizer.fit_on_texts(TRAIN_DATA[TRAIN_DATA['predict_sold'] == 1]['description'])
num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])
print(num_words)

tokenizer = Tokenizer(num_words=num_words)
print('fitting 2')
tokenizer.fit_on_texts(TRAIN_DATA[TRAIN_DATA['predict_sold'] == 1]['description'])

print('texts to seqs')
docs = tokenizer.texts_to_sequences(TRAIN_DATA['description'])

maxlen = 500

print('padding')
docs = pad_sequences(sequences=docs, maxlen=maxlen)

fitting 1
115154
fitting 2
texts to seqs
padding


MemoryError: 

In [16]:
def transform_to_sequences_and_pad(text_row, max_len=500):
    doc = tokenizer.texts_to_sequences(text_row)
    return pad_sequences(sequences=doc, maxlen=max_len)

In [17]:
del data

In [19]:
TRAIN_HEAD = TRAIN_DATA.head()

In [20]:
TRAIN_HEAD['sentimental'] = TRAIN_HEAD['description'].apply(lambda row: transform_to_sequences_and_pad(row))

/home/ubuntu/anaconda3/envs/data-ninja/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [35]:
th = TRAIN_HEAD.head(1)['sentimental'].iloc[0]

In [45]:
th[45]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   